## Regresión con Redes Neuronales (parte 2)

Se intentará predecir el costo de seguro médico para personas con base en diferentes parámetros

Se utilizará el dataset, disponible públicamente, llamado [Medical Cost dataset](https://www.kaggle.com/mirichoi0218/insurance) que se encuentra disponible en Kaggle y en [GitHub](https://github.com/stedy/Machine-Learning-with-R-datasets/blob/master/insurance.csv).


In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Lectura de los datos
df = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv")

In [ ]:
# Inspeccionar el conjunto de datos (dataset)
df.head()

Se convertirá las columnas categóricas utilizando one-hot encoding. Para ello, se utilizará [`get_dummies()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) de Pandas. 


In [ ]:
# Turn all categories into numbers
df_one_hot = pd.get_dummies(df)
df_one_hot.head() # view the converted columns

Se separará los datos en atributos y etiquetas



In [ ]:
# Crear X, y
X = df_one_hot.drop("charges", axis=1)
y = df_one_hot["charges"]

In [ ]:
X.head()

In [ ]:
y.head()

Se creará conjuntos de entrenamiento y prueba utilizando [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) de Scikit-Learn.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.2, 
                                                    random_state = 42)

Se construirá y entrenará el modelo

In [ ]:
tf.random.set_seed(42)

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(1),
                             tf.keras.layers.Dense(1)
])

model.compile(loss = tf.keras.losses.mae,
              optimizer = tf.keras.optimizers.SGD(),
              metrics = ['mae'])

historia = model.fit(X_train, y_train, epochs=100, verbose=0)

In [ ]:
# Gráfico de la función de pérdida
pd.DataFrame(historia.history).plot()

plt.ylabel("pérdida")
plt.xlabel("épocas");

In [ ]:
# Verificar los resultados del modelo (pérdida/MAE)
model.evaluate(X_test, y_test)

El modelo no se comporta bien. Se realizará 3 modificaciones:

- Incrementar el número de capas a 3
- Incrementar el número de unidades en cada capa (excepto en la capa de salida).
- Cambiar el optimizador (de SGD a Adam).

In [ ]:
tf.random.set_seed(42)

model_2 = tf.keras.Sequential([
                               tf.keras.layers.Dense(100), # 100 neuronas
                               tf.keras.layers.Dense(10),  # 10 neuronas
                               tf.keras.layers.Dense(1)    # 1 neurona (para la salida)
])

model_2.compile(loss = tf.keras.losses.mae,
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['mae'])

history = model_2.fit(X_train, y_train, epochs=100, verbose=0)

Hay una mejora: el error se reduce a aproximadamente la mitad

Para muchos problemas el optimizador de [Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) es una buena alternativa. Una discusión más profunda se puede encontrar en [*A Recipe for Training Neural Networks*](http://karpathy.github.io/2019/04/25/recipe/). 

In [ ]:
# Gráfico de la función de pérdida
pd.DataFrame(history.history).plot()

plt.ylabel("pérdida")
plt.xlabel("épocas");

In [ ]:
# Evaluación del nuevo modelo
model_2.evaluate(X_test, y_test)

In [ ]:
# Intentar entrenar por un poco más de épocas: 100 épocas más
history_2 = model_2.fit(X_train, y_train, epochs=100, verbose=0)

In [ ]:
# Evaluar el modelo entrenado por un total de 200 épocas
model_2_loss, insurance_model_2_mae = model_2.evaluate(X_test, y_test)

model_2_loss, insurance_model_2_mae

Entrenando 100 épocas más, se ha reducido el error en un 10%

In [ ]:
# Fundión de pérdida
pd.DataFrame(history_2.history).plot()

plt.ylabel("pérdida")
plt.xlabel("época"); # NOTA: solo se muestra los últimos 100 valores